# Data Sampling

## Setup

In [1]:
import sys

!{sys.executable} -m pip install --quiet --user --upgrade pandas==1.*
!{sys.executable} -m pip install --quiet --user --upgrade -r requirements.txt

In [2]:
from goodies import *
import pandas as pd

## Data Collection

In [3]:
from dcollect import plugins

modules = {'http': plugins.fasthttp()}
headers = None

### YouTube (United States)

Initial setup. Be sure to have your API key ready. For details on how to obtain an API key, read [YouTube Data API Overview, Introduction: Before you start](https://developers.google.com/youtube/v3/getting-started#before-you-start).

In [4]:
from dcollect import api_youtube as youtube
from dcollect import api_youtubei as youtubei

# This key is for testing ONLY. DO NOT release to the public!
api_key_testing = None
api_key = api_key_testing or input('YouTube Data API Key: ')

#### Search

In [5]:
count = 200
keyword = ''

##### STEP 1  API Object Creation

In [6]:
# create a YouTube API object
youtube_o = youtube.api(
    modules = modules,
    headers = headers,
    key = api_key
)

# create a YouTube Internals API object
youtubei_o = youtubei.api(
    modules = modules,
    headers = headers
)

##### STEP 2  Data Collection

In [7]:
from dcollect.utils.thread import threading, thread
from dcollect.utils.log import log

# set logging level
log.enable(level = log.levels.WARNING)


df_trending = df_from_json(
    youtube_o.video.trending(
        count = count
    )
)

df_channels = None
df_ads = None

thread.start([
    threading.Thread(
        # - channels
        target = lambda: \
            globals().update(
                df_channels = df_from_json(
                    youtube_o.channel.info(
                        id = df_trending['creator.id']
                    )
                )
            )
    ),
    threading.Thread(
        # - ad placements
        target = lambda: \
            globals().update(
                df_ads = df_from_json(
                    youtubei_o.ad.placements(
                        id = df_trending['id']
                    )
                )
            )
    )
])
thread.join()

##### STEP 3  Data Cleaning


In [8]:
# - ads (filter)
def filter_has_ad(ads):
    return not ads == None 
def filter_has_ad_beginning(ads):
    if ads == None:
        return False
    for ad in ads:
        if ad['kind'] == youtubei.resource.ad.kinds.START:
            return True
    return False
# - * (filter)
def drop_common(df, df_other, *args, **kwargs):
    return df.drop(columns = df.columns & df_other.columns, *args, **kwargs)

# - search
df_trending.set_index(['id', 'creator.id'], inplace = True)
# - channels
df_channels = df_channels.add_prefix('creator.')
df_channels.set_index(['creator.id'], inplace = True)
# - ads
df_ads.set_index(['id'], inplace = True)
df_ads['has_ad'] = df_ads['ads'].apply(filter_has_ad)
df_ads['has_ad_at_beginning'] = \
        df_ads['ads'].apply(filter_has_ad_beginning)
df_ads.drop('ads', axis = 'columns', inplace = True)

In [19]:
# - search (with details)
df_trending_details = df_trending.copy()
# - ads
df_trending_details = df_trending_details.merge(
    df_ads, 
    right_index = True, 
    left_on = 'id', 
    copy = False
)
# - channels
df_trending_details_channels = df_channels

##### STEP 4  Data Inspection


In [10]:
# take a brief look at our data
df_report(df_trending_details, name = 'Trending')
df_report(df_trending_details_channels, name = 'Trending (Channels)')

---

# Results - Trending

## Data Preview

title  \
id          creator.id                                           
R0R0gSE1jmM UCw1SQ6QRRtfAhrN_cjkrOgA  lazarbeam skin is BROKEN   
            UCw1SQ6QRRtfAhrN_cjkrOgA  lazarbeam skin is BROKEN   
            UCw1SQ6QRRtfAhrN_cjkrOgA  lazarbeam skin is BROKEN   
            UCw1SQ6QRRtfAhrN_cjkrOgA  lazarbeam skin is BROKEN   
g5TxSM16S6Q UCw1SQ6QRRtfAhrN_cjkrOgA   first game with MY SKIN   

                                                                            description  \
id          creator.id                                                                    
R0R0gSE1jmM UCw1SQ6QRRtfAhrN_cjkrOgA  the lazarbeam skin has a few bugs...\n\nCODE L...   
            UCw1SQ6QRRtfAhrN_cjkrOgA  the lazarbeam skin has a few bugs...\n\nCODE L...   
            UCw1SQ6QRRtfAhrN_cjkrOgA  the lazarbeam skin has a few bugs...\n\nCODE L...   
            UCw1SQ6QRRtfAhrN_cjkrOgA  the lazarbeam skin has a few bugs...\n\nCODE L...   
g5TxSM16S6Q UCw1SQ6QRRtfAhrN_cjkrOgA  LazarBeam X Fortnite MARCH 4TH - CODE LAZAR \n...   

                                                          time  \
id          creator.id                                           
R0R0gSE1jmM UCw1SQ6QRRtfAhrN_cjkrOgA 2021-03-09 03:06:47+00:00   
            UCw1SQ6QRRtfAhrN_cjkrOgA 2021-03-09 03:06:47+00:00   
            UCw1SQ6QRRtfAhrN_cjkrOgA 2021-03-09 03:06:47+00:00   
            UCw1SQ6QRRtfAhrN_cjkrOgA 2021-03-09 03:06:47+00:00   
g5TxSM16S6Q UCw1SQ6QRRtfAhrN_cjkrOgA 2021-03-03 12:22:32+00:00   

                                              length                tags  \
id          creator.id                                                     
R0R0gSE1jmM UCw1SQ6QRRtfAhrN_cjkrOgA 0 days 00:08:15            [gaming]   
            UCw1SQ6QRRtfAhrN_cjkrOgA 0 days 00:08:15            [gaming]   
            UCw1SQ6QRRtfAhrN_cjkrOgA 0 days 00:08:15            [gaming]   
            UCw1SQ6QRRtfAhrN_cjkrOgA 0 days 00:08:15            [gaming]   
g5TxSM16S6Q UCw1SQ6QRRtfAhrN_cjkrOgA 0 days 00:09:59  [gaming, fortnite]   

                                     category  stats.like  stats.dislike  \
id          creator.id                                                     
R0R0gSE1jmM UCw1SQ6QRRtfAhrN_cjkrOgA   Gaming      214199           1559   
            UCw1SQ6QRRtfAhrN_cjkrOgA   Gaming      214199           1559   
            UCw1SQ6QRRtfAhrN_cjkrOgA   Gaming      214199           1559   
            UCw1SQ6QRRtfAhrN_cjkrOgA   Gaming      214199           1559   
g5TxSM16S6Q UCw1SQ6QRRtfAhrN_cjkrOgA   Gaming      402680           4977   

                                      stats.comment  stats.view video.quality  \
id          creator.id                                                          
R0R0gSE1jmM UCw1SQ6QRRtfAhrN_cjkrOgA        14148.0     3840764            HD   
            UCw1SQ6QRRtfAhrN_cjkrOgA        14148.0     3840764            HD   
            UCw1SQ6QRRtfAhrN_cjkrOgA        14148.0     3840764            HD   
            UCw1SQ6QRRtfAhrN_cjkrOgA        14148.0     3840764            HD   
g5TxSM16S6Q UCw1SQ6QRRtfAhrN_cjkrOgA        37761.0     8506267            HD   

                                     creator.title  \
id          creator.id                               
R0R0gSE1jmM UCw1SQ6QRRtfAhrN_cjkrOgA     LazarBeam   
            UCw1SQ6QRRtfAhrN_cjkrOgA     LazarBeam   
            UCw1SQ6QRRtfAhrN_cjkrOgA     LazarBeam   
            UCw1SQ6QRRtfAhrN_cjkrOgA     LazarBeam   
g5TxSM16S6Q UCw1SQ6QRRtfAhrN_cjkrOgA     LazarBeam   

                                                                    creator.description  \
id          creator.id                                                                    
R0R0gSE1jmM UCw1SQ6QRRtfAhrN_cjkrOgA  THIS CHANNEL FEATURES AUSTRALIAN MAN\nFor Spon...   
            UCw1SQ6QRRtfAhrN_cjkrOgA  THIS CHANNEL FEATURES AUSTRALIAN MAN\nFor Spon...   
            UCw1SQ6QRRtfAhrN_cjkrOgA  THIS CHANNEL FEATURES AUSTRALIAN MAN\nFor Spon...   
            UCw1SQ6QRRtfA

## Stats

,length,stats.like,stats.dislike,stats.comment,stats.view,creator.stats.follower,creator.stats.view,creator.stats.post
count,224,2.240000e+02,224.00000,224.000000,2.240000e+02,2.240000e+02,2.240000e+02,224.000000
mean,0 days 00:09:03.214285714,1.122353e+05,1857.50000,13507.928571,2.466097e+06,5.963271e+06,2.543321e+09,2299.071429
std,0 days 00:08:27.451266582,1.629592e+05,2652.20721,39329.942919,3.515777e+06,1.126673e+07,7.772647e+09,7504.823009
min,0 days 00:00:23,3.787000e+03,110.00000,504.000000,2.719440e+05,8.150000e+03,7.057505e+06,6.000000
25%,0 days 00:03:09.250000,2.378825e+04,392.00000,2114.500000,7.449148e+05,8.350000e+05,7.787239e+07,68.500000
50%,0 days 00:05:58.500000,5.627450e+04,715.00000,4760.500000,1.282617e+06,2.090000e+06,3.019321e+08,326.500000
75%,0 days 00:11:31.750000,1.057530e+05,1564.00000,10137.000000,2.452179e+06,6.627500e+06,1.589453e+09,1223.500000
max,0 days 00:37:48,1.021220e+06,10751.00000,293380.000000,2.089441e+07,7.430000e+07,5.583855e+10,53319.000000


---

##### STEP 5  Data Archiving (Cumulative)


In [11]:
pickle_proto = 3

pickle_fname = 'dsamples/youtube_trending.pkl'
pickle_fname_channels = 'dsamples/youtube_trending_channels.pkl'

df_update_pickle(df_trending_details, pickle_fname, proto = pickle_proto)
df_update_pickle(df_trending_details_channels, pickle_fname_channels, proto = pickle_proto)

# verify that we saved the correct data
df_trending_details_verify = pd.read_pickle(pickle_fname)
df_report(df_trending_details_verify, name = 'Trending (Verification)')
df_trending_details_channels_verify = pd.read_pickle(pickle_fname)
df_report(df_trending_details_channels_verify, name = 'Trending (Channels) (Verification)')

---

# Results - Trending (Verification)

## Data Preview

title  \
id          creator.id                                                                    
-hAztPfxZEk UCtj45MepAoKxZoyR_Mnt86Q                FIRST DAY IN OUR NEW HOME! (HECTIC)   
            UCtj45MepAoKxZoyR_Mnt86Q                FIRST DAY IN OUR NEW HOME! (HECTIC)   
            UCtj45MepAoKxZoyR_Mnt86Q                FIRST DAY IN OUR NEW HOME! (HECTIC)   
            UCtj45MepAoKxZoyR_Mnt86Q                FIRST DAY IN OUR NEW HOME! (HECTIC)   
1XtcaLagLEo UCCj956IF62FbT7Gouszaj9w  Meghan accuses royals of 'perpetuating falseho...   

                                                                            description  \
id          creator.id                                                                    
-hAztPfxZEk UCtj45MepAoKxZoyR_Mnt86Q  LAST VIDEO: \r\n\r\n*MAKE SURE OUR POST NOTIFI...   
            UCtj45MepAoKxZoyR_Mnt86Q  LAST VIDEO: \r\n\r\n*MAKE SURE OUR POST NOTIFI...   
            UCtj45MepAoKxZoyR_Mnt86Q  LAST VIDEO: \r\n\r\n*MAKE SURE OUR POST NOTIFI...   
            UCtj45MepAoKxZoyR_Mnt86Q  LAST VIDEO: \r\n\r\n*MAKE SURE OUR POST NOTIFI...   
1XtcaLagLEo UCCj956IF62FbT7Gouszaj9w  Subscribe and 🔔 to OFFICIAL BBC YouTube 👉 http...   

                                                          time  \
id          creator.id                                           
-hAztPfxZEk UCtj45MepAoKxZoyR_Mnt86Q 2021-03-09 03:38:11+00:00   
            UCtj45MepAoKxZoyR_Mnt86Q 2021-03-09 03:38:11+00:00   
            UCtj45MepAoKxZoyR_Mnt86Q 2021-03-09 03:38:11+00:00   
            UCtj45MepAoKxZoyR_Mnt86Q 2021-03-09 03:38:11+00:00   
1XtcaLagLEo UCCj956IF62FbT7Gouszaj9w 2021-03-04 06:58:02+00:00   

                                              length  \
id          creator.id                                 
-hAztPfxZEk UCtj45MepAoKxZoyR_Mnt86Q 0 days 00:14:23   
            UCtj45MepAoKxZoyR_Mnt86Q 0 days 00:14:23   
            UCtj45MepAoKxZoyR_Mnt86Q 0 days 00:14:23   
            UCtj45MepAoKxZoyR_Mnt86Q 0 days 00:14:23   
1XtcaLagLEo UCCj956IF62FbT7Gouszaj9w 0 days 00:10:45   

                                                                                   tags  \
id          creator.id                                                                    
-hAztPfxZEk UCtj45MepAoKxZoyR_Mnt86Q  [queen Naija, Medicine, Queen, Royalty Squad, ...   
            UCtj45MepAoKxZoyR_Mnt86Q  [queen Naija, Medicine, Queen, Royalty Squad, ...   
            UCtj45MepAoKxZoyR_Mnt86Q  [queen Naija, Medicine, Queen, Royalty Squad, ...   
            UCtj45MepAoKxZoyR_Mnt86Q  [queen Naija, Medicine, Queen, Royalty Squad, ...   
1XtcaLagLEo UCCj956IF62FbT7Gouszaj9w  [British TV, British TV Shows, Watch UK TV Onl...   

                                             category  stats.like  \
id          creator.id                                              
-hAztPfxZEk UCtj45MepAoKxZoyR_Mnt86Q    Entertainment     24363.0   
            UCtj45MepAoKxZoyR_Mnt86Q    Entertainment     24363.0   
            UCtj45MepAoKxZoyR_Mnt86Q    Entertainment     24363.0   
            UCtj45MepAoKxZoyR_Mnt86Q    Entertainment     24363.0   
1XtcaLagLEo UCCj956IF62FbT7Gouszaj9w  News & Politics      2356.0   

                                      stats.dislike  stats.comment  \
id          creator.id                                               
-hAztPfxZEk UCtj45MepAoKxZoyR_Mnt86Q          400.0         1450.0   
            UCtj45MepAoKxZoyR_Mnt86Q          400.0         1450.0   
            UCtj45MepAoKxZoyR_Mnt86Q          400.0         1450.0   
            UCtj45MepAoKxZoyR_Mnt86Q          400.0         1450.0   
1XtcaLagLEo UCCj956IF62FbT7Gouszaj9w          876.0         2861.0   

                                      stats.view video.quality creator.title  \
id          creator.id                                                         
-hAztPfxZEk UCtj45MepAoKxZoyR_Mnt86Q    360642.0            HD  Royal Family   
            UCtj45MepAoKxZoyR_Mnt86Q    360642.0            HD  Royal Family   
            UCtj4

## Stats

,length,stats.like,stats.dislike,stats.comment,stats.view,creator.stats.follower,creator.stats.view,creator.stats.post
count,664,6.640000e+02,664.000000,651.000000,6.640000e+02,6.640000e+02,6.640000e+02,664.000000
mean,0 days 00:09:51.944277108,1.220250e+05,2884.320783,12511.391705,2.516563e+06,5.488461e+06,1.983159e+09,3718.052711
std,0 days 00:09:30.577349023,1.907659e+05,11265.946403,30623.037625,3.439644e+06,8.854426e+06,4.958621e+09,9220.233843
min,0 days 00:00:14,3.260000e+02,110.000000,339.000000,9.052100e+04,8.150000e+03,7.057505e+06,6.000000
25%,0 days 00:03:04,1.732200e+04,356.000000,2182.000000,7.029980e+05,7.510000e+05,1.092367e+08,126.000000
50%,0 days 00:06:11,5.016300e+04,697.000000,4513.000000,1.295399e+06,1.880000e+06,4.881651e+08,703.000000
75%,0 days 00:11:55,1.267690e+05,2705.750000,10029.000000,2.416874e+06,8.370000e+06,1.770375e+09,1647.000000
max,0 days 00:49:23,1.516146e+06,142630.000000,293380.000000,2.089441e+07,7.430000e+07,5.583855e+10,53319.000000


---